In [1]:
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import pandas as pd
import json
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import Embeddings
import pickle
import ast
from time import time

In [2]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, embedder):
        self.embedder = embedder
    def __call__(self, input: Documents) -> Embeddings:
        return self.embedder.embed_documents(input)

In [3]:
# !!! BELOW TO CHANGE !!! 
DATA_NAME = 'mtssquad'
TABLE_VERSION = 'v1'
DB_VERSION = 'v1'

EMBEDDING_MODEL_PATH = '../../../../nlp_models/intfloat/multilingual-e5-small'
MODEL_KWARGS = {'device': 'cuda'}
ENCODE_KWARGS = {'normalize_embeddings': False, 'prompt': 'passage: '}
CHROMA_KWARGS = {"hnsw:space": "l2"}
# !!! ABOVE TO CHANGE !!!

LOAD_DIR = f'../data/{DATA_NAME}/tables/{TABLE_VERSION}'
DATASET_PATH = f'{LOAD_DIR}/chunked_docs.csv'
SAVE_DIR = f"../data/{DATA_NAME}/dbs/{DB_VERSION}"
DENSE_DB_SAVE_PATH = f'{SAVE_DIR}/densedb'
DB_LOG_PATH = f'{SAVE_DIR}/operation_info.json' 

#### Preparing

In [4]:
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_PATH,
    model_kwargs=MODEL_KWARGS,
    encode_kwargs=ENCODE_KWARGS 
)
ef = MyEmbeddingFunction(embeddings)

/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-/.nlp_venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name ../../../../nlp_models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [5]:
client = chromadb.PersistentClient(path=DENSE_DB_SAVE_PATH)
collection = client.get_or_create_collection(name=DATA_NAME,  metadata=CHROMA_KWARGS, 
                                             embedding_function=ef)

In [6]:
df = pd.read_csv(DATASET_PATH, sep=';')
df['metadata'] = df['metadata'].map(lambda v: ast.literal_eval(v)) 
for i in range(df.shape[0]):
    df['metadata'][i]['prev_chunk_id'] = 'None'
    df['metadata'][i]['next_chunk_id'] = 'None'

#### Vectorizing 

In [7]:
vectorize_t_start = time()

collection.add(
    documents=df['chunks'].to_list(),
    metadatas=df['metadata'].to_list(),
    ids=list(map(lambda v: str(v['chunk_id']), df['metadata'].to_list()))
)

VECTORIZE_ELAPSED_TIME = round(time() - vectorize_t_start, 5)

In [8]:
collection.count()

9028

#### Saving Log

In [9]:
with open(DB_LOG_PATH, 'w') as fd:
    fd.write(json.dumps({
        "data_name": DATA_NAME,
        "table_version": TABLE_VERSION,
        "db_version": DB_VERSION, "model_name": EMBEDDING_MODEL_PATH,
        "encode_kwargs": ENCODE_KWARGS, "chroma_kwargs": CHROMA_KWARGS,
        "vectorize_elapsed_sec_time": VECTORIZE_ELAPSED_TIME}, indent=1))